*01-PYTORCH Conv2D*

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms



transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])



train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3),      
            nn.ReLU(),
            nn.MaxPool2d(2, 2),       

            nn.Conv2d(32, 64, 3),     
            nn.ReLU(),
            nn.MaxPool2d(2, 2)        
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),                
            nn.Linear(64 * 5 * 5, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    epoch_loss = running_loss / len(train_loader)
    print(f"{epoch+1}/{num_epochs} Loss: {epoch_loss:.4f}")



model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


print(f"Test  accuracy :{100 * correct / total:.2f}%")


1/5 Loss: 0.1352
2/5 Loss: 0.0443
3/5 Loss: 0.0300
4/5 Loss: 0.0212
5/5 Loss: 0.0151
Test  accuracy :98.90%


*02-Tensorflow Conv2D*

In [6]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Flatten , Conv2D , MaxPooling2D , Dense
from tensorflow.keras import models

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)).astype('float32')/ 255.0
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)).astype('float32')/ 255.0


model = models.Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)


print(f" test accuracy : {test_acc*100:.2f}%")


c:\Users\pr\anaconda3\envs\torch\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 29s 29ms/step - accuracy: 0.8828 - loss: 0.3985 - val_accuracy: 0.9817 - val_loss: 0.0603
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 24s 29ms/step - accuracy: 0.9837 - loss: 0.0525 - val_accuracy: 0.9885 - val_loss: 0.0474
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 24s 29ms/step - accuracy: 0.9905 - loss: 0.0313 - val_accuracy: 0.9893 - val_loss: 0.0388
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 24s 28ms/step - accuracy: 0.9925 - loss: 0.0223 - val_accuracy: 0.9903 - val_loss: 0.0361
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 25s 29ms/step - accuracy: 0.9945 - loss: 0.0171 - val_accuracy: 0.9918 - val_loss: 0.0364
313/313 - 2s - 7ms/step - accuracy: 0.9918 - loss: 0.0266
 test accuracy : 99.18%
